In [23]:
'''
Author - Pratik Ramdasi
Date - 03/24/2017
Project Title - Tensorflow MNIST practice

'''
## MNIST Classification using Tensorflow - CNN model
# import libraries
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [24]:
# Get input MNIST data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [42]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape = 28*28)
n_classes= 10 # Total number of classes (0-9 digits)
dropout = 0.75 # dropout, prob to keep units

# TF graph input
x = tf.placeholder(tf.float32, shape=[None, n_input])
y = tf.placeholder(tf.float32, shape=[None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Weight variable with noisy data, to prevent 0 gradients
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    tf.Variable(initial)
    
# Bias variable with init positive bias to avoid 'dead neurons'
def bias_variable(shape):
    init = tf.constant(0.1, shape=shape)
    tf.Variable(init)

## Build CNN model
    # Convolution layer
def conv2d(x, W, b):
    x = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

    # Pooling layer
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    # to apply tha layer, reshape the image
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    # First Convolution Layer
    h_conv1 = conv2d(x_image, weights['wc1'], biases['bc1'])
    h_conv1 = max_pool_2x2(h_conv1)
    
    # Second Convolution Layer
    h_conv2 = conv2d(h_conv1, weights['wc2'], biases['bc2'])
    h_conv2 = max_pool_2x2(h_conv2)
    
    # Fully Connected layer
        # reshape conv2 layer output to fit to fully connected layer
    h_fc1 = tf.reshape(h_conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    h_fc1 = tf.add(tf.matmul(h_fc1, weights['wd1']), biases['bd1'])
    h_fc1 = tf.nn.relu(h_fc1)
        # apply dropout
    h_fc1_drop = tf.nn.dropout(h_fc1, dropout)
    
    # Outputs, class prediction
    out = tf.add(tf.matmul(h_fc1_drop, weights['out']), biases['out'])
    return out

In [43]:
# Define Weights and Biases for each layer
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs
    'out': tf.Variable(tf.random_normal([1024, n_classes]))   
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate the model
predictions = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(predictions, tf.float32))

# Initialize the variables
init = tf.global_variables_initializer()

In [44]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # keep trainint until max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        if step % display_step == 0:
            # calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + \
                    ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"
    
    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy: ", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.})

Iter 1280, Minibatch Loss= 24152.828125, Training Accuracy= 0.25781
Iter 2560, Minibatch Loss= 11504.333984, Training Accuracy= 0.49219
Iter 3840, Minibatch Loss= 5089.477539, Training Accuracy= 0.71875
Iter 5120, Minibatch Loss= 4809.693359, Training Accuracy= 0.71875
Iter 6400, Minibatch Loss= 6484.457520, Training Accuracy= 0.69531
Iter 7680, Minibatch Loss= 1773.120483, Training Accuracy= 0.88281
Iter 8960, Minibatch Loss= 1377.451904, Training Accuracy= 0.89062
Iter 10240, Minibatch Loss= 2203.036621, Training Accuracy= 0.85156
Iter 11520, Minibatch Loss= 2612.071777, Training Accuracy= 0.86719
Iter 12800, Minibatch Loss= 2210.321289, Training Accuracy= 0.89062
Iter 14080, Minibatch Loss= 2050.126953, Training Accuracy= 0.92188
Iter 15360, Minibatch Loss= 1373.216553, Training Accuracy= 0.90625
Iter 16640, Minibatch Loss= 2168.783203, Training Accuracy= 0.86719
Iter 17920, Minibatch Loss= 2514.775635, Training Accuracy= 0.89844
Iter 19200, Minibatch Loss= 1466.734863, Training Acc